<a href="https://colab.research.google.com/github/wangyashuu/notebooks/blob/main/Tutorial_04_(huggingface)_Learning_Equality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


https://www.kaggle.com/competitions/learning-equality-curriculum-recommendations

*

# Download

In [1]:
# !pip install \
#     --extra-index-url=https://pypi.nvidia.com \
#     cudf-cu12==24.4.* dask-cudf-cu12==24.4.* cuml-cu12==24.4.* \
#     cugraph-cu12==24.4.* cuspatial-cu12==24.4.* cuproj-cu12==24.4.* \
#     cuxfilter-cu12==24.4.* cucim-cu12==24.4.* pylibraft-cu12==24.4.* \
#     raft-dask-cu12==24.4.* cuvs-cu12==24.4.*

!pip install transformers[torch]

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'learning-equality-curriculum-recommendations:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F39585%2F4786639%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240528%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240528T163732Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D095763b91bd541bab3fc45a1a54066dbb9a4fa717b4eeaaf48364f6b70fb74bb0b18d9b89de4ef400080b32a6506b7ad55a49809a06b5ccc2907ce9afe5b1d3b67fe4bf915e936eb465f4f2c58fd00d7a22a989217cce05ce00dea6cdbb388f5d77775feab15fe991e9a68e35744dc098cccf4c3c1d3d9b44c40bcc51c770ce8a8a6c727f72d1133b743715ba7f21fe1ec306660043d7269678b9fe406004108643c96d6d3951495b1572f8459e65196da45f0afc2c35d741ace2896cefd6443d0de3722e5c182edd525790ffe2d9da01b2f184f3c12ef90341e87d0c4d27a8048b67e482c917f3fc35d1027aec67427123b8dda8b7bd0df7d45d65bc3cc378c,lecr0297train:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2787243%2F4857804%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240528%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240528T163732Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4da855f949828e23170f326c842e9785a51a154a45ef84bec851f3a2edc26c47bf7495e3d865cff1a7b61a1af30352f29a7a7d416f794c0e791f4be22b9f6607d495484e9669e5eee94e446bc92c96adb71aa001c0d58ff94ad4646cc3ba74a0e00b36b7671517ba71f2d9b8f03edf85ca1c0fba610951d5b110f2792ec92b020ceb1619cd7fd91a421a14a94afa6a19cd0d66c86d133a9f039b6ac53c16a9a1eccaaf7519e4d1e5215212c20db558187d9f0ba103e4df796e29f55b10557c16b52411bc8e18e0454d79826b85b88eca280835ddc96f3799eb51f8e711563b8d8a94eb73f126dca3c79f528052328b1d28607c4d5a7e66ff420d415be01f1955,learning-equality:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2939693%2F5076707%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240528%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240528T163732Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D77e4ce9fa92283aae06ec808ab07d5b5e46570050056396ba93bf4dff61ab05ddb5539f8ee062643d52f05b3fd795747f214056273cb6c32be09e6ded32157220b0997443d67ba5a1253abab497e8f2bdde156a86f65d24aee5056cabfa80db14aac47c6a7eaa01d55be79cdc24c2111993bf9292bebb5194a7de3c10930e2d80b99e881e4e05c3e26d61c8e9e7a73a42cbfdd7d914c76d4268f0d6c5e1023fb96d3e42959ee74aa4f57922af7fba3bbc0d8239b09c3a0698eb1cd57a31152ffce8745729e4ed6101096f4436e692b811f05a4b71cf0d311053ad028341b294b308d3b93660378e433ecc1c02ae68ddde1f87fbfd801cc06c1800f93f4dad307'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 266113590 bytes downloaded
Downloaded and uncompressed: learning-equality-curriculum-recommendations
[==================================================] 1790030265 bytes downloaded

# Prepare

In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import Trainer, TrainingArguments
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data.dataloader import default_collate
from sklearn.model_selection import KFold

# import evaluate
# import cupy as cp
# from cuml.metrics import pairwise_distances
# from cuml.neighbors import NearestNeighbors

# import matplotlib.pyplot as plt
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, RobustScaler
# import seaborn as sns

In [ ]:
data_path = KAGGLE_INPUT_PATH + '/learning-equality-curriculum-recommendations/'
device = "cuda" if torch.cuda.is_available() else "cpu"

class Config:
    seed = 2024
    k_fold = 5
    n_epochs = 10

In [ ]:
df_content = pd.read_csv(data_path + "content.csv")
df_correlations = pd.read_csv(data_path + "correlations.csv")
df_topics = pd.read_csv(data_path + "topics.csv")

df_content.shape, df_correlations.shape, df_topics.shape

In [ ]:
df_content.head()

In [ ]:
df_correlations.head()

In [ ]:
df_topics.head()

# Data Exploration

## Missing Values

In [ ]:
def na_status(df, params = dict()):
    na_counts = pd.isna(df).sum()
    n = len(df.index)
    df_status = pd.DataFrame(
        dict(na_counts=na_counts, na_percents=na_counts / n, **params),
        index=df.columns
    )

    df_status[df_status['na_counts'] > 0].sort_values('na_counts', ascending=False)
    return df_status

In [ ]:
na_status(df_content)

In [ ]:
na_status(df_correlations)

In [ ]:
na_status(df_topics)

## Mislabeled Values

In [ ]:
df_content.describe()

In [ ]:
df_correlations.describe()

In [ ]:
df_topics.describe()

In [ ]:
(df_content.groupby(['title']).nunique() > 1).sum()

# Modelling

## Model and Dataset

In [ ]:
lm_name = 'dbmdz/bert-tiny-historic-multilingual-cased'

tokenizer = AutoTokenizer.from_pretrained(lm_name)

In [ ]:
class WrapperModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, **inputs):
        model = self.model
        anchor, positive, negative = inputs['anchor'], inputs['positive'], inputs['negative']
        anchor_output = model(**anchor)
        positive_output = model(**positive)
        negative_output = model(**negative)

        return (anchor_output.last_hidden_state.mean(dim=1),
            positive_output.last_hidden_state.mean(dim=1),
            negative_output.last_hidden_state.mean(dim=1))


triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2, eps=1e-7)


class TripletTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        anchor, positive, negative = outputs
        loss = triplet_loss(anchor, positive, negative)
        return (loss, outputs) if return_outputs else loss


def accuracy_triplet(anchor, positive, negative, margin = 0):
    positive_distance = torch.dist(anchor, positive, p=2)
    negative_distance = torch.dist(anchor, negative, p=2)
    pred = (negative_distance - positive_distance - margin) > 0
    return pred.sum() / len(anchor)

In [ ]:
def tokenize_fields(df, fields, padding=True, truncation=True, return_tensors="pt"):
    text = ''
    for f in fields:
        text += df[f]

    return tokenizer(
        text.fillna('').to_list(),
        padding=padding,
        truncation=truncation,
        return_tensors=return_tensors)


def rand_not_in(arr, excludes):
    while True:
        idx = np.random.randint(len(arr))
        if arr[idx] not in excludes:
            return idx


class CorrelationDataset(Dataset):
    def __init__(self, df_correlations, df_topics, df_content, tokenizer):
        ids_topic = df_correlations['topic_id']
        token_topics = tokenize_fields(
            df_topics.set_index('id').loc[ids_topic],
             ['title', 'description'])
        ids_content = list(set(df_correlations['content_ids'].str.cat(sep=' ').split()))
        token_content = tokenize_fields(
            df_content.set_index('id').loc[ids_content],
             ['title', 'description'])
        self.df_correlations = df_correlations
        self.ids_topics = pd.Index(ids_topic)
        self.token_topics = token_topics
        self.ids_content = pd.Index(ids_content)
        self.token_content = token_content

    def __len__(self):
        return len(self.df_correlations)

    def __getitem__(self, idx):
        tid, cids = self.df_correlations.iloc[idx]
        cids = cids.split()
        a_idx = self.ids_topics.get_loc(tid)
        p_idx = self.ids_content.get_loc(cids[np.random.randint(len(cids))])
        n_idx = rand_not_in(self.ids_content, cids)
        anchor = {k: v[a_idx] for k, v in self.token_topics.items()}
        positive = {k: v[p_idx] for k, v in self.token_content.items()}
        negative = {k: v[n_idx] for k, v in self.token_content.items()}
        return dict(anchor=anchor, positive=positive, negative=negative)


## Training Loop

In [ ]:
kf = KFold(n_splits=Config.k_fold, random_state=Config.seed, shuffle=True)

def compute_metrics(eval_pred):
    outputs, labels = eval_pred
    return dict(accuracy=accuracy_triplet(*outputs))

limit_size = 10000
df_filtered = df_correlations[:limit_size]

models = []
for i, (train_index, test_index) in enumerate(kf.split(df_filtered)):
    params = dict(df_topics=df_topics, df_content=df_content, tokenizer=tokenizer)
    dataset_train = CorrelationDataset(df_filtered.iloc[train_index], **params)
    dataset_test = CorrelationDataset(df_filtered.iloc[test_index], **params)

    training_args = TrainingArguments(
        output_dir= KAGGLE_WORKING_PATH + '/' + str(i),
        num_train_epochs=1, # Config.n_epochs,
        evaluation_strategy="epoch",
        remove_unused_columns=False,
        logging_steps=20,
        learning_rate=1e-4)

    model = AutoModel.from_pretrained(lm_name)
    wrapper_model = WrapperModel(model)
    trainer = TripletTrainer(
        model=wrapper_model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_test,
        data_collator=default_collate,
        compute_metrics=compute_metrics)
    trainer.train()
    models.append(model)
    break

In [ ]:
https://docs.rapids.ai/api/cuml/stable/api/#neighbors
https://github.com/rapidsai/cuml/blob/main/notebooks/nearest_neighbors_demo.ipynb

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
content.csv  correlations.csv  sample_submission.csv  topics.csv


In [ ]:
def distance_by_ids(ids):
    topic_idx = topics.loc[topics['id'] == ids[0]].index.item()
    content_idx = content.loc[content['id'] == ids[1]].index.item()

    return 1 - F.cosine_similarity(
        torch.tensor(topic_embeddings[[topic_idx]]),
        torch.tensor(content_embeddings[[content_idx]])
    )

# Useful Links

* what did in feature extraction pipeline https://huggingface.co/tasks/feature-extraction

* [Tips and Recommendations from Hosts](https://www.kaggle.com/code/jamiealexandre/tips-and-recommendations-from-hosts/notebook)